In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sqlalchemy import create_engine 
import os
from dotenv import load_dotenv
load_dotenv("../.env.postgres")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
conn = psycopg2.connect(
    database = "amazon_uk",
    user = os.environ["POSTGRES_USER"],
    password = os.environ["POSTGRES_PASSWORD"]
)

We first analyze the words used in title of top 5% most bought products vs. top 5% least bought products

In [3]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" DESC LIMIT 100000;', conn)
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15920\3787557939.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" DESC LIMIT 100000;', conn)


,index,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,205639,B001MZV3OO,Catsan Hygiene Cat Litter 20L,https://m.media-amazon.com/images/I/71bURZaHfF...,https://www.amazon.co.uk/dp/B001MZV3OO,4.7,31204,14.99,False,50000,Pet Supplies
1,1557617,B004OCO20E,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...,https://m.media-amazon.com/images/I/81K5VbVsCx...,https://www.amazon.co.uk/dp/B004OCO20E,4.6,44511,22.22,False,50000,Grocery
2,1252283,B07V2N4SJY,Amazon Brand – Mama Bear Sensitive Unscented B...,https://m.media-amazon.com/images/I/61lBVjA7jr...,https://www.amazon.co.uk/dp/B07V2N4SJY,4.7,32294,12.60,False,40000,Health & Personal Care
3,995347,B08WR5CPNB,Maybelline New York Lash Sensational Sky High ...,https://m.media-amazon.com/images/I/41vtg+FgTx...,https://www.amazon.co.uk/dp/B08WR5CPNB,4.4,52463,8.75,False,30000,Make-up
4,1557618,B01JS6YLQK,"Regina Blitz Household Towel, 560 Super-Sized ...",https://m.media-amazon.com/images/I/81+y0+Frsp...,https://www.amazon.co.uk/dp/B01JS6YLQK,4.8,40648,14.99,True,30000,Grocery


In [4]:
# check if a word has non alphabet character
def check_word(word):
    return all(w in "abcdefghijklmnopqrstuvwxyz" for w in word.lower())

In [5]:
most_bought_words = " ".join(title for title in df.title).lower().split(" ")
most_bought_words = [b for b in most_bought_words if (b not in stop_words) and (check_word(b))]
print(f"There are {len(most_bought_words)} words that in best seller category")

There are 1320915 words that in best seller category


In [6]:
most_bought_words_df = pd.DataFrame({"word": most_bought_words})
most_bought_words_df = most_bought_words_df["word"].value_counts().reset_index().rename({"index":"word"}, axis = 1)
most_bought_words_df.head()

,word,count
0,x,11195
1,pack,9708
2,hair,8468
3,kids,6934
4,set,6159


In [7]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" LIMIT 100000;', conn)
least_bought_words = " ".join(title for title in df.title).lower().split(" ")
least_bought_words = [b for b in least_bought_words if (b not in stop_words) and (check_word(b))]
print(f"There are {len(least_bought_words)} words that in best seller category")

C:\Users\Admin\AppData\Local\Temp\ipykernel_15920\2293965054.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 ORDER BY "boughtInLastMonth" LIMIT 100000;', conn)


There are 1296162 words that in best seller category


In [8]:
least_bought_words_df = pd.DataFrame({"word": least_bought_words})
least_bought_words_df = least_bought_words_df["word"].value_counts().reset_index().rename({"index":"word"}, axis = 1)
least_bought_words_df.head()

,word,count
0,optical,10889
1,golf,10637
2,laser,9446
3,player,9063
4,lens,9059


In [9]:
only_most_bought_words_df = most_bought_words_df[~most_bought_words_df["word"].isin(set(least_bought_words))].reset_index().drop("index", axis = 1)
only_most_bought_words_df.head()

,word,count
0,curling,272
1,wig,243
2,nappy,215
3,gummies,187
4,toothpaste,185


In [10]:
only_least_bought_words_df = least_bought_words_df[~least_bought_words_df["word"].isin(set(most_bought_words))].reset_index().drop("index", axis = 1)
only_least_bought_words_df.head()

,word,count
0,optique,4129
1,eeeone,3766
2,opticals,3253
3,keneddng,1400
4,lasereinheit,1348


We also analyze the words used in title of top rated products (5 stars) (not in under 1 stars product)

In [11]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 5;', conn)
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15920\2649471446.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 5;', conn)


,index,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,74,B09LYZM9RJ,"Fenton SHFS10W Active Subwoofer Speaker, 10"" f...",https://m.media-amazon.com/images/I/51jduec8al...,https://www.amazon.co.uk/dp/B09LYZM9RJ,5.0,1,169.00,False,0,Hi-Fi Speakers
1,267,B0BMB7B56J,Orange Portable Bluetooth Speaker Delivering S...,https://m.media-amazon.com/images/I/81R946t1Uf...,https://www.amazon.co.uk/dp/B0BMB7B56J,5.0,8,275.00,False,0,Hi-Fi Speakers
2,391,B0BHXJ72JT,Think Gizmos Portable Wireless Bluetooth Speak...,https://m.media-amazon.com/images/I/81sAggCZAJ...,https://www.amazon.co.uk/dp/B0BHXJ72JT,5.0,6,24.95,False,0,Hi-Fi Speakers
3,436,B0C8N8TMNM,"EnjoyNest Sound Bars for TV, 30 Watts Small So...",https://m.media-amazon.com/images/I/51jSjJI8nK...,https://www.amazon.co.uk/dp/B0C8N8TMNM,5.0,7,45.99,False,0,Hi-Fi Speakers
4,466,B0CDBV94S6,MAJORITY Bluetooth Sound Bar for TV with Wirel...,https://m.media-amazon.com/images/I/51UeNz2znO...,https://www.amazon.co.uk/dp/B0CDBV94S6,5.0,2,99.95,False,0,Hi-Fi Speakers


In [12]:
most_stars_words = " ".join(title for title in df.title).lower().split(" ")
most_stars_words = [b for b in most_stars_words if (b not in stop_words) and (check_word(b))]
print(f"There are {len(most_stars_words)} words that in best seller category")

There are 1250079 words that in best seller category


In [13]:
most_stars_words_df = pd.DataFrame({"word": most_stars_words})
most_stars_words_df = most_stars_words_df["word"].value_counts().reset_index().rename({"index":"word"}, axis = 1)
most_stars_words_df.head()

,word,count
0,gift,7733
1,x,7643
2,kids,6269
3,black,6251
4,set,5399


In [14]:
df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 0;', conn)
least_stars_words = " ".join(title for title in df.title).lower().split(" ")
least_stars_words = [b for b in least_stars_words if (b not in stop_words) and (check_word(b))]
print(f"There are {len(least_stars_words)} words that in best seller category")

C:\Users\Admin\AppData\Local\Temp\ipykernel_15920\2409724413.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM amazon_uk_2023 WHERE stars = 0;', conn)


There are 16465336 words that in best seller category


In [15]:
least_stars_words_df = pd.DataFrame({"word": least_stars_words})
least_stars_words_df = least_stars_words_df["word"].value_counts().reset_index().rename({"index":"word"}, axis = 1)
least_stars_words_df.head()

,word,count
0,women,153392
1,jacket,130458
2,men,124804
3,casual,105058
4,sleeve,104863


In [16]:
only_most_stars_words_df = most_stars_words_df[~most_stars_words_df["word"].isin(set(least_stars_words))].reset_index().drop("index", axis = 1)
only_most_stars_words_df.head()

,word,count
0,nnme,30
1,winodws,30
2,homei,16
3,gigabti,15
4,tunderbolt,14


In [17]:
only_least_stars_words_df = least_stars_words_df[~least_stars_words_df["word"].isin(set(most_stars_words))].reset_index().drop("index", axis = 1)
only_least_stars_words_df.head()

,word,count
0,nhnkb,7489
1,rtdgcv,5031
2,qwuveds,4216
3,optique,4159
4,folosafenar,3743


In [18]:
conn.close()

Save these datasets to Postgres

In [19]:
conn_string = os.environ.get("POSTGRES_CONN")
conn = create_engine(conn_string) 

In [20]:
only_most_bought_words_df.to_sql('amazon_uk_2023_most_bought_words', conn, if_exists= 'replace') 

595

In [21]:
only_least_bought_words_df.to_sql('amazon_uk_2023_least_bought_words', conn, if_exists= 'replace') 

78

In [22]:
only_most_stars_words_df.to_sql('amazon_uk_2023_most_stars_words', conn, if_exists= 'replace') 

725

In [23]:
only_least_stars_words_df.to_sql('amazon_uk_2023_least_stars_words', conn, if_exists= 'replace') 

500